In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Reading from sockets")
    .master( "local[*]")
    .getOrCreate()
)

spark

In [2]:
spark.conf.set("spark.sql.adaptive.enabled",False)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled",False)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

In [3]:
spark.sparkContext.defaultParallelism

16

In [4]:
df_1 = spark.range(4,200,2)
df_2 = spark.range(2,200,4)

In [5]:
df_1.rdd.getNumPartitions()
df_2.rdd.getNumPartitions()

16

In [6]:
df_3=  df_1.repartition(5)
df_4=  df_2.repartition(7)

In [7]:
display(df_3.rdd.getNumPartitions())
df_4.rdd.getNumPartitions()

5

7

In [8]:
df_joined = df_3.join(df_4, on="id")

In [9]:
df_sum = df_joined.selectExpr("sum(id) as total_sum")

In [10]:
df_sum.show()

+---------+
|total_sum|
+---------+
|     4998|
+---------+



In [11]:
df_sum.explain()

== Physical Plan ==
*(6) HashAggregate(keys=[], functions=[sum(id#0L)])
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#182]
   +- *(5) HashAggregate(keys=[], functions=[partial_sum(id#0L)])
      +- *(5) Project [id#0L]
         +- *(5) SortMergeJoin [id#0L], [id#2L], Inner
            :- *(2) Sort [id#0L ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(id#0L, 200), ENSURE_REQUIREMENTS, [id=#166]
            :     +- Exchange RoundRobinPartitioning(5), REPARTITION_BY_NUM, [id=#165]
            :        +- *(1) Range (4, 200, step=2, splits=16)
            +- *(4) Sort [id#2L ASC NULLS FIRST], false, 0
               +- Exchange hashpartitioning(id#2L, 200), ENSURE_REQUIREMENTS, [id=#173]
                  +- Exchange RoundRobinPartitioning(7), REPARTITION_BY_NUM, [id=#172]
                     +- *(3) Range (2, 200, step=4, splits=16)




In [12]:
'''
    At each shuffle (end of stage) spark write the data and the next stage will read them
    If its not involed repartition or coalesce then at each shuffle will create n default shuffle partitions (200 default)
    In case of failure the job will take the data from the last stage shuffle write and continue the next steps
'''

'\n    At each shuffle (end of stage) spark write the data and the next stage will read them\n    If its not involed repartition or coalesce then at each shuffle will create n default shuffle partitions (200 default)\n    In case of failure the job will take the data from the last stage shuffle write and continue the next steps\n'

In [13]:
# rdds

df_1.rdd

MapPartitionsRDD[5] at javaToPython at NativeMethodAccessorImpl.java:0